In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

Целта на задачата беше да подобрим резултата от 0.82 на тест, направен на лекции.

In [16]:
#функцийка за дропване на колони, исках просто да пробвам нещо
def dropper(set, *args):
    for arg in args:
        set = set.drop(arg, axis=1)
    return set

train = pd.read_csv('/home/string/dev/machine-learning/supervised-learning/train.csv', index_col='PassengerId')
train = dropper(train, 'Cabin', 'Ticket')

Оправяме набързо колоната Sex и попълваме празните клетки във Embarked.

In [17]:
train['Sex'] = train['Sex'].apply(lambda x: 0 if x == 'male' else 1)
train['Embarked'] = train.Embarked.fillna('S')

После extract-ваме титлите от имената на хората и създаваме нова колона Title. След това полълваме празните клетки в Age със средната стойност на съответната титлова група.

In [18]:
train['Title'] = train.Name.str.extract('([A-Za-z]+)\.', expand=False)
train.loc[train.Title == 'Mlle', 'Title'] = 'Miss'
train.loc[train.Title == 'Mme', 'Title'] = 'Mrs'
train.loc[train.Title == 'Ms', 'Title'] = 'Miss'
rare_titles = ['Dr', 'Rev', 'Col', 'Major', 'Countess', 'Don', 'Jonkheer', 'Capt', 'Lady', 'Sir']
train.Title = train.Title.replace(rare_titles, 'Rare')

age_by_title = train.groupby('Title').Age.mean()

train.loc[train.Age.isnull() & (train.Title == 'Mr'), 'Age'] = age_by_title['Mr']
train.loc[train.Age.isnull() & (train.Title == 'Mrs'), 'Age'] = age_by_title['Mrs']
train.loc[train.Age.isnull() & (train.Title == 'Miss'), 'Age'] = age_by_title['Miss']
train.loc[train.Age.isnull() & (train.Title == 'Master'), 'Age'] = age_by_title['Master']
train.loc[train.Age.isnull() & (train.Title == 'Rare'), 'Age'] = age_by_title['Rare']

Encode-ваме Титлите и градовете на хората.

In [19]:
from sklearn.preprocessing import LabelEncoder
titleEncoder = LabelEncoder()
titles = ['Mr', 'Miss', 'Mrs', 'Rare', 'Master']
titleEncoder.fit(titles)

train['Title'] = titleEncoder.transform(train.Title)

In [20]:
embarkEncoder = LabelEncoder()
embark = ['Q', 'S', 'C']
embarkEncoder.fit(embark)

train['Embarked'] = embarkEncoder.transform(train.Embarked)

Създаваме FamilySize колона и премахваме Parch и SibSp.

In [21]:
train['FamilySize'] = train.Parch + train.SibSp + 1
train = dropper(train, 'Parch', 'SibSp')

In [9]:
X = dropper(train, 'Name', 'Survived')
y = train['Survived']

x_train, x_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=0)

Накрая тренираме и .... малко подобрение, но не можах да намеря начин за по-добър резултат. Каквото друго пробвах беше под този.

In [24]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=9, random_state=11)

model.fit(x_train, y_train)
model.score(x_test, y_test)

0.85201793721973096